In [1]:
!nvidia-smi

Sun Jun 26 11:22:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! pip install -q datasets
! pip install torch==1.6.0
! pip install transformers

     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 101 kB 12.4 MB/s 
     |████████████████████████████████| 140 kB 72.6 MB/s 
     |████████████████████████████████| 212 kB 59.3 MB/s 
     |████████████████████████████████| 1.1 MB 61.5 MB/s 
     |████████████████████████████████| 596 kB 61.3 MB/s 
     |████████████████████████████████| 127 kB 76.7 MB/s 
     |████████████████████████████████| 271 kB 72.5 MB/s 
     |████████████████████████████████| 144 kB 76.9 MB/s 
     |████████████████████████████████| 94 kB 3.2 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 748.8 MB 18 kB/s 

In [4]:
import pandas as pd
import numpy as np
import os
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/train.csv')
eval_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dataset/kaggle//us-patent/test.csv')

In [5]:
df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
df.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1:A47; TEXT2:abatement of pollution; ANC1:...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1:A47; TEXT2:act of abating; ANC1:abatement
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1:A47; TEXT2:active catalyst; ANC1:abatement
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1:A47; TEXT2:eliminating process; ANC1:aba...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1:A47; TEXT2:forest region; ANC1:abatement


In [6]:
from datasets import Dataset,DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [7]:
from transformers import AutoTokenizer
import torch

new_model = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(new_model)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [8]:
tokenizer.tokenize("A platypus is an ornithorhynchus anatinus.")

['a',
 'pl',
 '##at',
 '##yp',
 '##us',
 'is',
 'an',
 'or',
 '##ni',
 '##thor',
 '##hy',
 '##nch',
 '##us',
 'ana',
 '##tin',
 '##us',
 '.']

In [9]:
tokenizer("A platypus is an ornithorhynchus anatinus.")

{'input_ids': [101, 1037, 20228, 4017, 22571, 2271, 2003, 2019, 2030, 3490, 27844, 10536, 12680, 2271, 9617, 7629, 2271, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [10]:
def tok_func(x): return tokenizer(x['input'])
tok_ds = ds.map(tok_func, batched = True)

Parameter 'function'=<function tok_func at 0x7fcc310d49e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/37 [00:00<?, ?ba/s]

In [11]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [12]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [13]:
from numpy.random import normal,seed,uniform
np.random.seed(2022)
train_and_val = tok_ds.train_test_split(0.25, seed = 2022)
train_and_val

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [14]:
eval_df['input'] = 'TEXT1:' + df.context + '; TEXT2:' + df.target + '; ANC1:' + df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched = True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36
})

In [16]:
def corr(x,y): return np.corrcoef(x,y)[0][1]
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

In [17]:
from transformers import Trainer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(new_model, ignore_mismatched_sizes=True, num_labels = 1)
trainer = Trainer(model,  
          train_dataset = train_and_val['train'],
          eval_dataset = train_and_val['test'], 
          tokenizer = tokenizer, 
          compute_metrics = corr_d)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

In [18]:
trainer.train();

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: context, anchor, id, input, target. If context, anchor, id, input, target are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 10260


Step,Training Loss
500,0.059300
1000,0.044600
1500,0.042000
2000,0.040800
2500,0.037600
3000,0.034200
3500,0.033300
4000,0.024100
4500,0.025000
5000,0.023500


Saving model checkpoint to tmp_trainer/checkpoint-500
Configuration saved in tmp_trainer/checkpoint-500/config.json
Model weights saved in tmp_trainer/checkpoint-500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-500/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-500/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1000
Configuration saved in tmp_trainer/checkpoint-1000/config.json
Model weights saved in tmp_trainer/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1000/tokenizer_config.json
Special tokens file saved in tmp_trainer/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to tmp_trainer/checkpoint-1500
Configuration saved in tmp_trainer/checkpoint-1500/config.json
Model weights saved in tmp_trainer/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in tmp_trainer/checkpoint-1500/tokenizer_config.json
Special tokens file saved in tmp_traine

In [19]:
prediction = trainer.predict(eval_ds).predictions.astype(float)
prediction

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: context, anchor, id, input, target. If context, anchor, id, input, target are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 8


array([[ 5.70825338e-01],
       [ 7.77962983e-01],
       [ 1.85291812e-01],
       [ 4.00071263e-01],
       [-8.29596072e-04],
       [ 1.47095501e-01],
       [ 1.52881995e-01],
       [ 1.79504290e-01],
       [ 5.74611068e-01],
       [ 2.40524083e-01],
       [ 5.53792238e-01],
       [ 2.47846216e-01],
       [ 5.53279400e-01],
       [ 5.39369702e-01],
       [ 2.58856025e-02],
       [ 1.86133776e-02],
       [ 1.59076944e-01],
       [ 4.35386360e-01],
       [ 4.85242307e-02],
       [ 5.53535745e-02],
       [ 1.17090061e-01],
       [ 5.64887822e-01],
       [ 3.06241333e-01],
       [ 1.88235775e-01],
       [ 5.55540919e-01],
       [ 6.19895875e-01],
       [ 2.18763933e-01],
       [ 6.17087901e-01],
       [ 9.66176391e-01],
       [ 1.03852851e-03],
       [ 1.80761203e-01],
       [ 2.24218994e-01],
       [ 2.64719333e-02],
       [-3.33409011e-03],
       [ 6.29185379e-01],
       [ 2.65370339e-01]])

In [20]:
pred = np.clip(prediction, 0, 1)
pred

array([[0.57082534],
       [0.77796298],
       [0.18529181],
       [0.40007126],
       [0.        ],
       [0.1470955 ],
       [0.15288199],
       [0.17950429],
       [0.57461107],
       [0.24052408],
       [0.55379224],
       [0.24784622],
       [0.5532794 ],
       [0.5393697 ],
       [0.0258856 ],
       [0.01861338],
       [0.15907694],
       [0.43538636],
       [0.04852423],
       [0.05535357],
       [0.11709006],
       [0.56488782],
       [0.30624133],
       [0.18823577],
       [0.55554092],
       [0.61989588],
       [0.21876393],
       [0.6170879 ],
       [0.96617639],
       [0.00103853],
       [0.1807612 ],
       [0.22421899],
       [0.02647193],
       [0.        ],
       [0.62918538],
       [0.26537034]])

In [21]:
import datasets
submission = datasets.Dataset.from_dict(
    {
       'id': eval_ds['id'],
       'score': pred 
    }
)
submission

Dataset({
    features: ['id', 'score'],
    num_rows: 36
})

In [22]:
submission.to_csv('distilbert_base.csv', index = False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1067